# Homework 2

# Set up

## Installing packages

In [14]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [15]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [16]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 11.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 67.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 37.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 17.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 8.58MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [17]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [18]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [20]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_VERTEX_API_KEY,
    temperature=0,
)

# from langchain_openai import ChatOpenAI
# DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
# llm = ChatOpenAI(
#     model="deepseek-chat",          # or "deepseek-reasoner"
#     api_key=DEEPSEEK_API_KEY,
#     base_url="https://api.deepseek.com/v1",
#     temperature=0,
# )

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content='' additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c9df2-867a-7262-bed4-1ec09083e8df-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': '2a02bcd3-82f0-47ea-8a0c-b8b26e4ababf', 'type': 'tool_call'}, {'name': 'search_facebook_users', 'args': {'q': 'Alice'}, 'id': '8a8c8642-0525-4ce1-9766-d2bbcb4473fd', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2852, 'output_tokens': 33, 'total_tokens': 2885, 'input_token_details': {'cache_read': 0}}


In [21]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
# await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
await tools[5].ainvoke({'person_id': 456})

[{'type': 'text',
  'text': '{"profile_id":456,"post_count":4,"total_likes":5,"liked_by":[4390,3622,7500,4269,8464],"engagement_score":1.25}',
  'id': 'lc_c7611b7e-ca39-425e-9a96-18b46aaa0d36'}]

In [40]:
import json
import re
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage

system_prompt = """You are an automated background verification engine.
Your objective is to compare the candidate's CV against social profiles and provide a verifiable JSON report.

PROCESS PIPELINE (STRICTLY REQUIRED):
1. Parse the provided CV for primary claims: Name, Current/Recent Companies, Degrees, Universities.
2. Execute 'search_linkedin_people' and 'search_facebook_users' with 'fuzzy'=True to locate account IDs. Broaden your search by avoiding overly restrictive constraints (like strict location).
3. EXTREMELY IMPORTANT: You MUST invoke 'get_linkedin_profile' and/or 'get_facebook_profile' utilizing the acquired IDs. NEVER construct your final answer merely off search metadata. Look deep into the experience arrays!
4. Compile the evidence to score the CV accurately.

SCORING ALGORITHM (BASE CV IS GROUND TRUTH):
- Authentic Match (Score 0.6 to 1.0): Core professional track (Employment / Major degree) extracted from social API matches the CV. Ignore internal structural logic errors in the CV (like date overlap), 'is_current' tag bugs in the API, or platform inconsistencies on trivial factors (e.g. hometown vs origin city).
- Proven Fabrication (Score 0.0 to 0.4): Evidence unequivocally confirms a completely falsified professional track (e.g., claiming Senior Manager at Top Tech, while confirmed real profile exhibits lower rank at unrelated entity, or severe academic discrepancies).
- Note on Name Clashes: If Facebook contradicts LinkedIn dramatically on a common name (e.g. Rahul Sharma, John Smith, Wei Zhang), assess the platform that holds higher professional alignment. Do not default score to fake due to noise.

OUTPUT REQUIREMENT (PURE JSON):
Return JSON strictly representing the conclusion schema:
{"analysis": "[Explain specific points compared, list verified versus refuted facts explicitly, state justification for the numerical score.]", "score": [numerical value]}
"""

def process_response_structure(content):
    if isinstance(content, str): return content
    if isinstance(content, list):
        return "".join(blk.get("text", "") if isinstance(blk, dict) else str(blk) for blk in content)
    return str(content)

scores = []
reports =[]

print("Initializing background verification pipeline...\n")

for cv in all_cvs:
    cv_name = cv["file"]
    cv_text = cv["text"]

    print("-" * 75)
    print(f"Audit Target Processed: {cv_name}")

    messages =[
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Verify this target payload:\n{cv_text}")
    ]

    final_score = 0.5
    final_rationale = "Timeout / System iteration bound met without extraction."

    try:
        # Increase iteration envelope slightly to guarantee get_profile finishes processing
        for i in range(8):
            response = llm_with_tools.invoke(messages)
            messages.append(response)

            if not response.tool_calls:
                # Execution node reached conclusion string
                output_blob = process_response_structure(response.content)
                break

            for tc in response.tool_calls:
                args = dict(tc["args"])
                fname = tc["name"]
                # Sanitize the tool injections context constraints
                if fname == "search_facebook_users":
                    args = {k: v for k, v in args.items() if k in["q", "limit", "fuzzy"]}
                elif fname == "search_linkedin_people":
                    args = {k: v for k, v in args.items() if k in["q", "location", "industry", "limit", "fuzzy"]}

                print(f"  [API Access] Invoked `{fname}` via payload: {args}")
                tool_func = next(t for t in tools if t.name == fname)
                tool_result = await tool_func.ainvoke(args)

                messages.append(ToolMessage(
                    tool_call_id=tc["id"],
                    name=fname,
                    content=str(tool_result)
                ))

        # Scope processing
        clean_text = output_blob.replace("```json", "").replace("```", "").strip()
        json_search = re.search(r'\{.*\}', clean_text, re.DOTALL)

        if json_search:
            try:
                dict_extract = json.loads(json_search.group(), strict=False)
                final_score = float(dict_extract.get("score", 0.5))
                final_rationale = str(dict_extract.get("analysis", ""))
            except json.JSONDecodeError:
                sm = re.search(r'"score"\s*:\s*([\d\.]+)', clean_text)
                if sm: final_score = float(sm.group(1))

        reports.append({
            "File": cv_name,
            "Evaluation": final_score,
            "Rationale Trace": final_rationale
        })

        print(f"  -> Rationale Tracking Logging:\n  |  {final_rationale}\n")
        print(f"  [Output Registered]: Verification Factor -> {final_score}")

    except Exception as e:
        print(f"[Fatal Processing Fault] Engine collapsed tracing {cv_name}: {e}\n")

    scores.append(final_score)

print("=" * 75)
print(f"Complete Routine Assessment Executed")
print(f"Output Submission Form Array: scores = {scores}")
print("=" * 75)

Initializing background verification pipeline...

---------------------------------------------------------------------------
Audit Target Processed: CV_1.pdf
  [API Access] Invoked `search_linkedin_people` via payload: {'q': 'John Smith', 'location': 'Singapore', 'fuzzy': True}
  [API Access] Invoked `get_linkedin_profile` via payload: {'person_id': 9}
  [API Access] Invoked `search_facebook_users` via payload: {'fuzzy': True, 'q': 'John Smith'}
  [API Access] Invoked `get_facebook_profile` via payload: {'user_id': 213}
  -> Rationale Tracking Logging:
  |  The candidate's CV claims 'John Smith' as a 'Marketing Professional' in 'Singapore', with experience as an 'Engineer at ByteDance' from '2020 – Present', and a 'Bachelor of Science (BSc) in Marketing' from 'McGill University', graduated '2009'. Social media verification was performed using LinkedIn and Facebook.

LinkedIn profile (ID 9) for 'John Smith' in 'Singapore' was found, with the headline 'Marketing Professional'. This prof

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [22]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [41]:
scores = [0.9, 0.9, 0.9, 0.1, 0.2] # Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


{'decisions': [1, 1, 1, 0, 0], 'correct': 5, 'total': 5, 'final_score': 1.0}
